In [20]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
# import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML
from sklearn.metrics import accuracy_score, classification_report


In [21]:
df = pd.read_csv("./data/modified/weather_arrivals_departures_merged_correct_tz.csv")
df["prev_flight_status"] = df["arrival_status"].shift(1)
df["prev_flight_status"].fillna("early", inplace=True)
df

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Taxi-In time (Minutes),Delay Carrier (Minutes)_x,Delay Weather (Minutes)_x,Delay National Aviation System (Minutes)_x,Delay Security (Minutes)_x,Delay Late Aircraft Arrival (Minutes)_x,scheduled_arrival_time,actual_arrival_time,date_x,date_time_x,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,timestamp_local_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,key_x,date_time_map_x,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes)_y,Delay Weather (Minutes)_y,Delay National Aviation System (Minutes)_y,Delay Security (Minutes)_y,Delay Late Aircraft Arrival (Minutes)_y,scheduled_departure_time,actual_departure_time,date_y,date_time_y,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,timestamp_local_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,key_y,date_time_map_y,arrival_status,prev_flight_status
0,B6,01/01/2020,116,JFK,08:22,08:08,75,62,-14,3,0,0,0,0,0,2020-01-01 08:22:00,2020-01-01 08:08:00,2020-01-01,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,2020-01-01 08:15:00,0.40,16,804,250,12.10,7.07,SYR,2020-01-01 08:15:00,SYR,07:07,07:06,75,62,-1,11,0,0,0,0,0,2020-01-01 07:07:00,2020-01-01 07:06:00,2020-01-01,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,2020-01-01 07:00:00,0.00,16,804,280,11.10,7.70,JFK,2020-01-01 07:00:00,early,early
1,OO,01/01/2020,3247,ORD,10:43,10:40,113,110,-3,4,0,0,0,0,0,2020-01-01 10:43:00,2020-01-01 10:40:00,2020-01-01,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,2020-01-01 10:45:00,0.60,16,804,255,14.90,6.70,SYR,2020-01-01 10:45:00,SYR,07:50,07:50,113,110,0,28,0,0,0,0,0,2020-01-01 08:50:00,2020-01-01 08:50:00,2020-01-01,2020-01-01 07:50:00,-12.20,25,-7.90,984,87,-6.10,2020-01-01 07:45:00,1.00,16,802,200,6.60,4.59,ORD,2020-01-01 07:45:00,ontime,early
2,9E,01/01/2020,5066,JFK,11:37,11:03,92,72,-34,5,0,0,0,0,0,2020-01-01 11:37:00,2020-01-01 11:03:00,2020-01-01,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,2020-01-01 11:30:00,0.60,16,804,265,12.60,7.75,SYR,2020-01-01 11:30:00,SYR,10:05,09:51,92,72,-14,20,0,0,0,0,0,2020-01-01 10:05:00,2020-01-01 09:51:00,2020-01-01,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,2020-01-01 10:00:00,0.70,16,804,270,11.60,7.70,JFK,2020-01-01 10:00:00,early,ontime
3,F9,01/01/2020,1214,MCO,14:39,14:23,164,156,-16,6,0,0,0,0,0,2020-01-01 14:39:00,2020-01-01 14:23:00,2020-01-01,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,2020-01-01 14:45:00,0.50,16,804,260,12.00,7.32,SYR,2020-01-01 14:45:00,SYR,11:55,11:47,164,156,-8,14,0,0,0,0,0,2020-01-01 11:55:00,2020-01-01 11:47:00,2020-01-01,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,2020-01-01 12:00:00,4.00,16,802,290,5.20,3.10,MCO,2020-01-01 12:00:00,early,early
4,OO,01/01/2020,3253,ORD,14:43,14:38,113,110,-5,5,0,0,0,0,0,2020-01-01 14:43:00,2020-01-01 14:38:00,2020-01-01,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,2020-01-01 14:45:00,0.50,16,804,260,12.00,7.32,SYR,2020-01-01 14:45:00,SYR,11:50,11:48,113,110,-2,25,0,0,0,0,0,2020-01-01 12:50:00,2020-01-01 12:48:00,2020-01-01,2020-01-01 11:50:00,-2.80,25,-5.70,982,58,1.70,2020-01-01 11:45:00,1.80,16,802,200,10.80,5.09,ORD,2020-01-01 11:45:00,ontime,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,12/31/2023,3719,ORD,16:44,17:18,124,157,34,7,0,0,34,0,0,2023-12-31 16:44:00,2023-12-31 17:18:00,2023-12-31,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,2023-12-31 16:45:00,0.00,14,804,270,2.72,2.72,SYR,2023-12-31 16:45:00,SYR,13:40,13:41,124,157,1,58,0,0,34,0,0,2023-12-31 14:4

In [22]:
def convert_to_timestamp(df):
    if "scheduled_departure_time" in df.columns:
        df["scheduled_departure_time"] = pd.to_datetime(df["scheduled_departure_time"])
        
    if "scheduled_arrival_time" in df.columns:
        df["scheduled_arrival_time"] = pd.to_datetime(df["scheduled_arrival_time"])
    
    if "actual_departure_time" in df.columns:
        df["actual_departure_time"] = pd.to_datetime(df["actual_departure_time"])
        
    if "actual_arrival_time" in df.columns:
        df["actual_arrival_time"] = pd.to_datetime(df["actual_arrival_time"])
    
    return df


In [23]:
# Adding 1hr to ORD to convert to EST from CST
# df.loc[df["Origin Airport"]=="ORD", "scheduled_departure_time"] += pd.Timedelta(hours=1)
# df.loc[df["Origin Airport"]=="ORD", "actual_departure_time"] += pd.Timedelta(hours=1)


In [24]:
# df["diff1"] = (df["scheduled_arrival_time"]-df["scheduled_departure_time"]).dt.total_seconds() // 60
# df["diff2"] = (df["actual_arrival_time"]-df["actual_departure_time"]).dt.total_seconds() // 60


# df[df["Scheduled Elapsed Time (Minutes)"] != df["diff"]][["Scheduled Arrival Time", "scheduled_arrival_time", "Scheduled departure time", "scheduled_departure_time", "Scheduled elapsed time (Minutes)", "Scheduled Elapsed Time (Minutes)", "diff"]]
# df[df["Scheduled Elapsed Time (Minutes)"] != df["diff1"]][["Scheduled Arrival Time", "scheduled_arrival_time", "Scheduled departure time", "scheduled_departure_time", "Scheduled elapsed time (Minutes)", "Scheduled Elapsed Time (Minutes)", "diff1"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff2"]) & (df["Actual Elapsed Time (Minutes)"] != 0)][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff2", "actual_departure_time", "actual_arrival_time"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0)][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff", "actual_departure_time", "actual_arrival_time"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0) & (df["Actual Elapsed Time (Minutes)"] == df["diff2"])][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff", "diff2", "actual_departure_time", "actual_arrival_time"]]
# df.loc[df["Scheduled Elapsed Time (Minutes)"] != df["diff1"], "scheduled_arrival_time"] += pd.Timedelta(days=1)
# df.loc[(df["Actual Elapsed Time (Minutes)"] != df["diff2"]) & (df["Actual Elapsed Time (Minutes)"] != 0), "actual_arrival_time"] += pd.Timedelta(days=1)

# df.drop(columns="diff1", inplace=True)
# df.drop(columns="diff2", inplace=True)
# df["diff2"] = df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0)]["diff"] + 1440



In [25]:
 # df[(df["Actual Elapsed Time (Minutes)"]==0)]

In [26]:
# pd.concat([incor, df[df["Actual elapsed time (Minutes)"]==0]]).reset_index(drop=True).to_csv("./data/modified/incorrect_flight_data.csv", index=False)

In [27]:
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] == 0)].to_csv("./data/modified/incorrect_flight_data.csv")
# df = df[~((df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] == 0))]
# df = df[~(df["Actual Elapsed Time (Minutes)"]==0)].reset_index(drop=True)
# df
# df.drop(columns="diff", inplace=True)
# df.to_csv("./data/modified/weather_arrivals_departures_merged_correct_tz.csv", index=False)

In [28]:
df = convert_to_timestamp(df)

In [29]:
numeric_cols = df.select_dtypes(include='number').columns
numeric_cols

Index(['Flight Number', 'Scheduled Elapsed Time (Minutes)',
       'Actual Elapsed Time (Minutes)', 'Arrival Delay (Minutes)',
       'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)_x',
       'Delay Weather (Minutes)_x',
       'Delay National Aviation System (Minutes)_x',
       'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
       'app_temp_x', 'clouds_x', 'dewpt_x', 'pres_x', 'rh_x', 'temp_x', 'uv_x',
       'vis_x', 'weather.code_x', 'wind_dir_x', 'wind_gust_spd_x',
       'wind_spd_x', 'Scheduled elapsed time (Minutes)',
       'Actual elapsed time (Minutes)', 'Departure delay (Minutes)',
       'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)_y',
       'Delay Weather (Minutes)_y',
       'Delay National Aviation System (Minutes)_y',
       'Delay Security (Minutes)_y', 'Delay Late Aircraft Arrival (Minutes)_y',
       'app_temp_y', 'clouds_y', 'dewpt_y', 'pres_y', 'rh_y', 'temp_y', 'uv_y',
       'vis_y', 'weather.code_y', 'wind_dir_y', 'wind_gus

In [30]:
cat_cols = df.select_dtypes(exclude='number').columns
cat_cols

Index(['Carrier Code', 'Date (MM/DD/YYYY)', 'Origin Airport',
       'Scheduled Arrival Time', 'Actual Arrival Time',
       'scheduled_arrival_time', 'actual_arrival_time', 'date_x',
       'date_time_x', 'timestamp_local_x', 'key_x', 'date_time_map_x',
       'Destination Airport', 'Scheduled departure time',
       'Actual departure time', 'scheduled_departure_time',
       'actual_departure_time', 'date_y', 'date_time_y', 'timestamp_local_y',
       'key_y', 'date_time_map_y', 'arrival_status', 'prev_flight_status'],
      dtype='object')

In [31]:
cat_drop_cols = ["Date (MM/DD/YYYY)", "Scheduled Arrival Time", "Actual Arrival Time", "date_x", 
             "date_time_x", "timestamp_local_x", "key_x", "date_time_map_x", "Destination Airport", 
             "Scheduled departure time", "Actual departure time",'date_y', 'date_time_y', 'timestamp_local_y',
             'key_y', 'date_time_map_y', "actual_arrival_time", "actual_departure_time"
            ]

numeric_drop_cols = ['Flight Number', 'Actual Elapsed Time (Minutes)',
                     'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)_x','Delay Weather (Minutes)_x', 
                     'Delay National Aviation System (Minutes)_x', 'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
                     'Scheduled elapsed time (Minutes)','Actual elapsed time (Minutes)', 'Departure delay (Minutes)','Taxi-Out time (Minutes)', 
                     'Delay Carrier (Minutes)_y','Delay Weather (Minutes)_y','Delay National Aviation System (Minutes)_y',
                     'Delay Security (Minutes)_y', 'Delay Late Aircraft Arrival (Minutes)_y']

In [32]:
model_df = df.drop(columns=cat_drop_cols + numeric_drop_cols).copy()
model_df

,Carrier Code,Origin Airport,Scheduled Elapsed Time (Minutes),Arrival Delay (Minutes),scheduled_arrival_time,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,scheduled_departure_time,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,arrival_status,prev_flight_status
0,B6,JFK,75,-14,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,0.40,16,804,250,12.10,7.07,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,0.00,16,804,280,11.10,7.70,early,early
1,OO,ORD,113,-3,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,0.60,16,804,255,14.90,6.70,2020-01-01 08:50:00,-12.20,25,-7.90,984,87,-6.10,1.00,16,802,200,6.60,4.59,ontime,early
2,9E,JFK,92,-34,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,0.60,16,804,265,12.60,7.75,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,0.70,16,804,270,11.60,7.70,early,ontime
3,F9,MCO,164,-16,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,4.00,16,802,290,5.20,3.10,early,early
4,OO,ORD,113,-5,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 12:50:00,-2.80,25,-5.70,982,58,1.70,1.80,16,802,200,10.80,5.09,ontime,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,ORD,124,34,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,0.00,14,804,270,2.72,2.72,2023-12-31 14:40:00,-2.40,100,-2.10,989,80,1.00,0.50,4,804,310,7.40,3.10,late,early
11776,MQ,ORD,106,102,2023-12-31 17:28:00,-4.50,100,-3.70,991,86,-1.70,0.00,15,804,355,2.04,2.04,2023-12-31 15:42:00,-2.30,100,-2.30,989,78,1.10,0.40,10,804,300,6.60,3.10,late,late
11777,B6,MCO,175,-36,2023-12-31 19:27:00,-3.40,100,-4.00,1000,86,-1.90,0.00,15,804,360,2.40,1.20,2023-12-31 16:32:00,18.10,80,8.80,1015,52,18.90,1.00,16,804,305,4.40,4.15,early,late
11778,UA,ORD,114,-10,2023-12-31 21:09:00,-4.30,100,-3.90,1000,88,-2.20,0.00,16,804,360,3.30,1.50,2023-12-31 19:15:00,-5.40,87,-2.30,993,82,0.40,0.00,13,804,315,10.70,7.03,early,early


In [33]:
numeric_mod_cols = model_df.select_dtypes(include="number").columns
cat_mod_cols = model_df.select_dtypes(exclude="number").columns

In [34]:
model_df

,Carrier Code,Origin Airport,Scheduled Elapsed Time (Minutes),Arrival Delay (Minutes),scheduled_arrival_time,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,scheduled_departure_time,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,arrival_status,prev_flight_status
0,B6,JFK,75,-14,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,0.40,16,804,250,12.10,7.07,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,0.00,16,804,280,11.10,7.70,early,early
1,OO,ORD,113,-3,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,0.60,16,804,255,14.90,6.70,2020-01-01 08:50:00,-12.20,25,-7.90,984,87,-6.10,1.00,16,802,200,6.60,4.59,ontime,early
2,9E,JFK,92,-34,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,0.60,16,804,265,12.60,7.75,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,0.70,16,804,270,11.60,7.70,early,ontime
3,F9,MCO,164,-16,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,4.00,16,802,290,5.20,3.10,early,early
4,OO,ORD,113,-5,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 12:50:00,-2.80,25,-5.70,982,58,1.70,1.80,16,802,200,10.80,5.09,ontime,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,ORD,124,34,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,0.00,14,804,270,2.72,2.72,2023-12-31 14:40:00,-2.40,100,-2.10,989,80,1.00,0.50,4,804,310,7.40,3.10,late,early
11776,MQ,ORD,106,102,2023-12-31 17:28:00,-4.50,100,-3.70,991,86,-1.70,0.00,15,804,355,2.04,2.04,2023-12-31 15:42:00,-2.30,100,-2.30,989,78,1.10,0.40,10,804,300,6.60,3.10,late,late
11777,B6,MCO,175,-36,2023-12-31 19:27:00,-3.40,100,-4.00,1000,86,-1.90,0.00,15,804,360,2.40,1.20,2023-12-31 16:32:00,18.10,80,8.80,1015,52,18.90,1.00,16,804,305,4.40,4.15,early,late
11778,UA,ORD,114,-10,2023-12-31 21:09:00,-4.30,100,-3.90,1000,88,-2.20,0.00,16,804,360,3.30,1.50,2023-12-31 19:15:00,-5.40,87,-2.30,993,82,0.40,0.00,13,804,315,10.70,7.03,early,early


In [35]:
from sklearn.preprocessing import OneHotEncoder

def get_ohe(df, col):
    ohe = OneHotEncoder(drop='first', handle_unknown='error', sparse_output=False, dtype='int')
    ohe.fit(df[[col]])
    temp_df = pd.DataFrame(data=ohe.transform(df[[col]]), columns=ohe.get_feature_names_out())
    df.drop(columns=[col], axis=1, inplace=True)
    df = pd.concat([df.reset_index(drop=True), temp_df], axis=1)
    return df

def add_custom_features(model_df):
    model_df['scheduled_departure_hour'] = model_df['scheduled_departure_time'].dt.hour
    model_df['scheduled_departure_day'] = model_df['scheduled_departure_time'].dt.day
    model_df['scheduled_departure_month'] = model_df['scheduled_departure_time'].dt.month
    model_df['scheduled_departure_weekday'] = model_df['scheduled_departure_time'].dt.weekday

    model_df['scheduled_arrival_hour'] = model_df['scheduled_arrival_time'].dt.hour
    model_df['scheduled_arrival_day'] = model_df['scheduled_arrival_time'].dt.day
    model_df['scheduled_arrival_month'] = model_df['scheduled_arrival_time'].dt.month
    model_df['scheduled_arrival_weekday'] = model_df['scheduled_arrival_time'].dt.weekday

    weather_features = ['app_temp', 'clouds', 'dewpt', 'pres', 'rh', 'temp', 'uv', 'vis','weather.code', 'wind_dir', 'wind_gust_spd', 'wind_spd']
    
    for feature in weather_features:
        model_df[f'{feature}_diff'] = model_df[f'{feature}_x'] - model_df[f'{feature}_y']

    # carrier_dummies = pd.get_dummies(model_df['Carrier Code'], prefix='carrier')
    # airport_dummies = pd.get_dummies(model_df['Origin Airport'], prefix='airport')


    # model_df = pd.concat([model_df, carrier_dummies, airport_dummies], axis=1)

    model_df['dep_time_day_prop'] = model_df['scheduled_departure_time'].dt.hour * 60 + model_df['scheduled_departure_time'].dt.minute
    model_df['dep_time_day_prop'] /= 1440  

    model_df['arr_time_day_prop'] = model_df['scheduled_arrival_time'].dt.hour * 60 + model_df['scheduled_arrival_time'].dt.minute
    model_df['arr_time_day_prop'] /= 1440  


    model_df['night_flight'] = model_df['scheduled_departure_time'].dt.hour.apply(lambda x: 1 if x >= 20 or x < 6 else 0)

    model_df['extreme_weather_temp_low'] = model_df['temp_x'].apply(lambda x: 1 if x < -10 else 0)
    model_df['extreme_weather_temp_high'] = model_df['temp_x'].apply(lambda x: 1 if x > 35 else 0)
    model_df['extreme_weather_wind'] = model_df['wind_gust_spd_x'].apply(lambda x: 1 if x > 50 else 0)
    
    model_df['time_of_day'] = model_df['scheduled_arrival_hour'].apply(categorize_hour)

    model_df["peak_zone"] = "peak"

    model_df.loc[((model_df['time_of_day']=="Morning") | (model_df['time_of_day']=="Early Morning") | (model_df['time_of_day']=="Afternoon")), "peak_zone"] = "off_peak"
    
    model_df["origin_airport"] = model_df["Origin Airport"]
    model_df["flight_status_prev"] = model_df["prev_flight_status"]
    
    categorical_cols = ["Origin Airport", 'peak_zone', "prev_flight_status"]
    for i in categorical_cols:
        model_df = get_ohe(model_df, i)

    # model_df = pd.get_dummies(model_df, columns=categorical_cols)
    bool_cols = model_df.select_dtypes(include=["bool"]).columns
    for i in bool_cols:
        model_df[i] *= 1
    
    return model_df

def categorize_hour(hour):
    if 0 <= hour <= 3:
        return 'Early Morning'
    elif 4 <= hour <= 7:
        return 'Morning'
    elif 8 <= hour <= 11:
        return 'Late Morning'
    elif 12 <= hour <= 15:
        return 'Afternoon'
    elif 16 <= hour <= 19:
        return 'Evening'
    elif 20 <= hour <= 23:
        return 'Night'





In [36]:
model_df = add_custom_features(model_df)
numeric_cols = model_df.select_dtypes(include=['number']).columns.tolist()
drop_cols = model_df.select_dtypes(exclude=['number']).columns.tolist()
drop_cols.append('Arrival Delay (Minutes)')
numeric_cols.remove('Arrival Delay (Minutes)')

In [38]:


X = model_df.drop(columns=drop_cols, axis=1)
y = model_df['arrival_status']

scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])


In [39]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
X

,Scheduled Elapsed Time (Minutes),app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,scheduled_departure_hour,scheduled_departure_day,scheduled_departure_month,scheduled_departure_weekday,scheduled_arrival_hour,scheduled_arrival_day,scheduled_arrival_month,scheduled_arrival_weekday,app_temp_diff,clouds_diff,dewpt_diff,pres_diff,rh_diff,temp_diff,uv_diff,vis_diff,weather.code_diff,wind_dir_diff,wind_gust_spd_diff,wind_spd_diff,dep_time_day_prop,arr_time_day_prop,night_flight,extreme_weather_temp_low,extreme_weather_temp_high,extreme_weather_wind,Origin Airport_MCO,Origin Airport_ORD,peak_zone_peak,prev_flight_status_late,prev_flight_status_ontime
0,-1.14,-1.38,0.82,-1.09,-1.54,0.10,-1.18,-0.46,0.37,0.32,0.46,1.42,1.41,-1.46,0.63,-1.00,0.01,0.35,-1.29,-0.90,0.31,0.28,0.91,1.06,1.26,-1.80,-1.69,-1.64,-0.52,-1.49,-1.69,-1.64,-0.52,0.07,0.11,-0.05,-0.91,-0.22,0.10,0.60,0.07,0.06,-0.39,0.34,0.10,-1.86,-1.50,-0.54,-0.11,-0.01,0.00,-0.47,-0.88,0.53,-0.66,-0.48
1,0.13,-1.26,0.82,-0.99,-1.41,0.10,-1.08,-0.32,0.37,0.32,0.51,2.25,1.24,-2.25,-1.32,-1.54,-1.51,1.31,-2.17,-0.33,0.31,0.25,0.08,-0.32,-0.09,-1.58,-1.69,-1.64,-0.52,-1.12,-1.69,-1.64,-0.52,1.35,1.67,0.85,0.77,-1.06,1.44,0.07,0.07,0.08,0.31,2.19,1.07,-1.49,-1.07,-0.54,-0.11,-0.01,0.00,-0.47,1.14,0.53,-0.66,-0.48
2,-0.57,-1.27,0.82,-1.01,-1.41,-0.01,-1.06,-0.32,0.37,0.32,0.63,1.57,1.71,-1.34,0.63,-1.34,0.15,-0.71,-1.18,-0.50,0.31,0.28,0.81,1.22,1.26,-1.14,-1.69,-1.64,-0.52,-0.93,-1.69,-1.64,-0.52,0.07,0.11,0.53,-0.98,0.62,0.11,0.27,0.07,0.06,-0.18,0.34,0.34,-1.22,-0.90,-0.54,-0.11,-0.01,0.00,-0.47,-0.88,0.53,-0.66,2.06
3,1.84,-1.21,0.82,-1.03,-1.28,-0.17,-1.01,-0.39,0.37,0.32,0.57,1.39,1.52,0.28,-1.32,-0.52,0.81,-1.45,0.25,1.38,0.31,0.25,1.02,-0.75,-0.74,-0.92,-1.69,-1.64,-0.52,-0.38,-1.69,-1.64,-0.52,-2.09,1.67,-0.64,-1.59,1.13,-1.77,-2.00,0.07,0.08,-0.39,1.81,1.81,-0.82,-0.35,-0.54,-0.11,-0.01,0.00,2.15,-0.88,-1.90,-0.66,-0.48
4,0.13,-1.21,0.82,-1.03,-1.28,-0.17,-1.01,-0.39,0.37,0.32,0.57,1.39,1.52,-1.47,-1.32,-1.33,-1.66,-0.23,-1.40,0.13,0.31,0.25,0.08,0.97,0.13,-0.70,-1.69,-1.64,-0.52,-0.38,-1.69,-1.64,-0.52,0.34,1.67,0.49,1.00,0.06,0.47,-0.54,0.07,0.08,0.35,0.39,1.11,-0.62,-0.34,-0.54,-0.11,-0.01,0.00,-0.47,1.14,-1.90,-0.66,-0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,0.50,-1.29,0.82,-0.80,-1.15,1.37,-1.30,-0.73,-0.28,0.32,0.68,-1.36,-0.52,-1.44,1.04,-0.98,-1.15,0.94,-1.46,-0.61,-3.98,0.28,1.23,-0.08,-0.74,-0.26,1.75,1.51,1.47,-0.01,1.75,1.50,1.47,0.19,-0.22,0.30,0.54,0.34,0.16,-0.00,2.66,0.06,-0.47,-1.10,0.19,-0.23,0.03,-0.54,-0.11,-0.01,0.00,-0.47,1.14,0.53,-0.66,-0.48
11776,-0.10,-1.22,0.82,-0.83,-1.28,1.26,-1.30,-0.73,0.05,0.32,1.64,-1.56,-0.82,-1.43,1.04,-1.00,-1.15,0.83,-1.45,-0.67,-1.83,0.28,1.12,-0.32,-0.74,-0.05,1.75,1.51,1.47,0.17,1.75,1.50,1.47,0.27,-0.22,0.29,0.46,0.34,0.15,0.07,1.36,0.06,0.31,-1.07,-0.05,-0.00,0.17,-0.54,-0.11,-0.01,0.00,-0.47,1.14,0.53,1.52,-0.48
11777,2.21,-1.13,0.82,-0.86,-0.10,1.26,-1.32,-0.73,0.05,0.32,1.69,-1.46,-1.19,0.26,0.41,0.09,0.73,-0.55,0.31,-0.33,0.31,0.28,1.17,-1.00,-0.28,0.17,1.75,1.51,1.47,0.54,1.75,1.50,1.47,-1.95,0.28,-1.27,-0.83,1.55,-2.29,-0.34,-0.19,0.06,0.31,-0.42,-0.72,0.18,0.53,-0.54,-0.11,-0.01,0.00,2.15,-0.88,0.53,1.52,-0.48
11778,0.17,-1.20,0.82,-0.85,-0.10,1.37,-1.35,-0.73,0.37,0.32,1.69,-1.19,-1.06,-1.69,0.63,-1.00,-0.86,1.04,-1.52,-0.90,-0.76,0.28,1.28,0.94,0.97,0.83,1.75,1.51,1.47,0.91,1.75,1.50,1.47,0.65,0.11,0.26,0.84,0.24,0.18,0.33,0.85,0.06,0.23,-1.79,-1.63,0.77,0.84,-0.54,-0.11,-0.01,0.00,-0.47,1.14,0.53,-0.66,-0.48


In [41]:
len(X_train.columns)

56

In [42]:

temp_df = pd.read_csv("./data/modified/initial_predictions_m2.csv")
# temp_df["scheduled_departure_time"] = pd.to_datetime(temp_df["scheduled_departure_time"])
# temp_df["date"] = temp_df["scheduled_departure_time"].dt.date
# temp_df = temp_df.groupby(["date"]).apply(lambda x: x.sort_values('Origin Airport')).reset_index(drop=True)
# model1_df = temp_df.iloc[::2]
# model1_df
temp_df

,Origin Airport,scheduled_departure_time,scheduled_arrival_time,Carrier Code,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,Scheduled Elapsed Time (Minutes)
0,JFK,2024-04-10 14:55:00,2024-04-10 16:21:00,DL,17.10,80,8.80,998.00,58,17.10,1.80,24.13,804,286,6.10,4.13,12.30,91,8.20,"1,015.00",76,12.30,1.50,24.13,804,96,4.36,3.60,86.00
1,MCO,2024-04-10 13:35:00,2024-04-10 16:25:00,B6,17.10,80,8.80,998.00,58,17.10,1.80,24.13,804,286,6.10,4.13,29.60,46,16.00,"1,009.50",44,29.50,4.20,24.13,803,143,8.63,5.86,170.00
2,ORD,2024-04-10 19:59:00,2024-04-10 22:52:00,AA,11.90,75,6.50,999.00,69,11.90,0.00,24.13,804,120,4.00,2.40,11.80,91,3.60,980.50,57,11.80,0.00,24.13,804,56,4.66,2.93,173.00
3,JFK,2024-04-11 14:55:00,2024-04-11 16:21:00,DL,18.20,87,11.00,986.00,63,18.20,1.70,17.02,804,146,18.00,11.86,13.10,95,9.90,"1,008.00",81,13.10,1.40,10.03,804,156,13.20,8.80,86.00
4,MCO,2024-04-11 13:35:00,2024-04-11 16:25:00,B6,18.20,87,11.00,986.00,63,18.20,1.70,17.02,804,146,18.00,11.86,31.80,58,19.50,"1,005.50",53,30.20,3.70,24.13,202,213,20.13,13.33,170.00
5,ORD,2024-04-11 19:59:00,2024-04-11 22:52:00,AA,17.20,89,11.90,981.50,71,17.20,0.00,2.72,301,150,16.10,10.80,9.00,80,5.20,972.50,77,9.00,0.00,8.38,804,306,15.86,10.40,173.00
6,JFK,2024-04-12 14:55:00,2024-04-12 16:21:00,DL,13.50,79,5.90,972.00,60,13.50,1.90,23.95,804,230,14.20,9.60,15.10,53,7.90,993.50,62,15.10,2.60,24.13,803,226,14.60,9.87,86.00
7,MCO,2024-04-12 13:35:00,2024-04-12 16:25:00,B6,13.50,79,5.90,972.00,60,13.50,1.90,23.95,804,230,14.20,9.60,24.00,1,10.30,"1,015.50",41,24.40,6.10,24.13,800,296,8.86,5.87,170.00
8,ORD,2024-04-12 19:59:00,2024-04-12 22:52:00,AA,8.90,80,3.00,974.50,66,8.90,0.00,0.13,804,240,13.95,9.36,9.70,10,0.80,990.50,54,9.70,0.00,24.13,801,293,11.40,7.60,173.00
9,JFK,2024-04-13 14:55:00,2024-04-13 16:21:00,DL,8.10,73,1.10,989.00,61,8.10,2.10,21.12,804,280,15.23,10.11,12.40,54,2.00,"1,005.20",49,12.40,2.50,24.13,803,280,13.63,9.04,86.00


In [43]:
def read_actual_m2_files(period):
    df = pd.read_csv(f"./data/modified/{period}_predictions_m2.csv")
    df = convert_to_timestamp(df)
    df.loc[(df["Origin Airport"]=="JFK")&(df["Carrier Code"]=="DL"), "Carrier Code"] = "9E"
    df["prev_flight_status"] = [["early", "ontime", "late"]]*len(df)
    df = df.explode("prev_flight_status")
    df = add_custom_features(df)
    num_cols = df.select_dtypes(include=["number"]).columns.tolist()
    scaler = StandardScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols])
    cat_df = df.select_dtypes(exclude=['number'])
    cat_df.loc[(df["origin_airport"]=="JFK")&(df["Carrier Code"]=="9E"), "Carrier Code"] = "DL"
    d_cols = cat_df.columns.tolist()
    df.drop(columns=d_cols, inplace=True)
    df["peak_zone_peak"] = 0
    df = df[X_train.columns]
    
    return df, cat_df

In [44]:
initial_df, initial_cat_df = read_actual_m2_files("initial")
final_df, final_cat_df = read_actual_m2_files("final")

In [45]:
initial_df

,Scheduled Elapsed Time (Minutes),app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,scheduled_departure_hour,scheduled_departure_day,scheduled_departure_month,scheduled_departure_weekday,scheduled_arrival_hour,scheduled_arrival_day,scheduled_arrival_month,scheduled_arrival_weekday,app_temp_diff,clouds_diff,dewpt_diff,pres_diff,rh_diff,temp_diff,uv_diff,vis_diff,weather.code_diff,wind_dir_diff,wind_gust_spd_diff,wind_spd_diff,dep_time_day_prop,arr_time_day_prop,night_flight,extreme_weather_temp_low,extreme_weather_temp_high,extreme_weather_wind,Origin Airport_MCO,Origin Airport_ORD,peak_zone_peak,prev_flight_status_late,prev_flight_status_ontime
0,-1.32,0.79,-0.05,0.53,1.20,-1.15,0.83,0.70,0.68,0.32,1.14,-1.29,-1.18,-0.56,1.12,0.06,1.19,1.33,-0.56,-0.26,0.48,0.32,-1.33,-1.52,-1.23,-0.59,-1.28,0.00,-1.28,-0.76,-1.28,0.00,-1.28,1.10,-1.17,0.38,-0.32,-1.73,1.11,0.82,0.40,-0.04,1.85,0.31,-0.07,-0.54,-0.77,0.00,0.00,0.00,0.00,-0.61,-0.76,0,-0.71,-0.71
1,-1.32,0.79,-0.05,0.53,1.20,-1.15,0.83,0.70,0.68,0.32,1.14,-1.29,-1.18,-0.56,1.12,0.06,1.19,1.33,-0.56,-0.26,0.48,0.32,-1.33,-1.52,-1.23,-0.59,-1.28,0.00,-1.28,-0.76,-1.28,0.00,-1.28,1.10,-1.17,0.38,-0.32,-1.73,1.11,0.82,0.40,-0.04,1.85,0.31,-0.07,-0.54,-0.77,0.00,0.00,0.00,0.00,-0.61,-0.76,0,-0.71,1.41
2,-1.32,0.79,-0.05,0.53,1.20,-1.15,0.83,0.70,0.68,0.32,1.14,-1.29,-1.18,-0.56,1.12,0.06,1.19,1.33,-0.56,-0.26,0.48,0.32,-1.33,-1.52,-1.23,-0.59,-1.28,0.00,-1.28,-0.76,-1.28,0.00,-1.28,1.10,-1.17,0.38,-0.32,-1.73,1.11,0.82,0.40,-0.04,1.85,0.31,-0.07,-0.54,-0.77,0.00,0.00,0.00,0.00,-0.61,-0.76,0,1.41,-0.71
3,0.71,0.79,-0.05,0.53,1.20,-1.15,0.83,0.70,0.68,0.32,1.14,-1.29,-1.18,1.70,-0.38,1.46,0.81,-1.12,1.77,1.13,0.48,0.32,-0.76,-0.59,-0.50,-0.96,-1.28,0.00,-1.28,-0.76,-1.28,0.00,-1.28,-1.26,0.39,-1.45,-0.00,0.65,-1.39,-1.15,0.40,-0.04,1.36,-1.45,-1.46,-1.02,-0.74,0.00,0.00,0.00,0.00,1.63,-0.76,0,-0.71,-0.71
4,0.71,0.79,-0.05,0.53,1.20,-1.15,0.83,0.70,0.68,0.32,1.14,-1.29,-1.18,1.70,-0.38,1.46,0.81,-1.12,1.77,1.13,0.48,0.32,-0.76,-0.59,-0.50,-0.96,-1.28,0.00,-1.28,-0.76,-1.28,0.00,-1.28,-1.26,0.39,-1.45,-0.00,0.65,-1.39,-1.15,0.40,-0.04,1.36,-1.45,-1.46,-1.02,-0.74,0.00,0.00,0.00,0.00,1.63,-0.76,0,-0.71,1.41
5,0.71,0.79,-0.05,0.53,1.20,-1.15,0.83,0.70,0.68,0.32,1.14,-1.29,-1.18,1.70,-0.38,1.46,0.81,-1.12,1.77,1.13,0.48,0.32,-0.76,-0.59,-0.50,-0.96,-1.28,0.00,-1.28,-0.76,-1.28,0.00,-1.28,-1.26,0.39,-1.45,-0.00,0.65,-1.39,-1.15,0.40,-0.04,1.36,-1.45,-1.46,-1.02,-0.74,0.00,0.00,0.00,0.00,1.63,-0.76,0,1.41,-0.71
6,0.79,-0.29,-1.03,-0.10,1.30,0.99,-0.40,-1.31,0.68,0.32,-1.58,-1.73,-1.70,-0.62,1.12,-0.77,-1.18,-0.13,-0.63,-1.03,0.48,0.32,-1.82,-1.46,-1.45,1.31,-1.28,0.00,-1.28,1.32,-1.28,0.00,-1.28,0.46,-1.34,0.92,1.72,0.50,0.43,0.60,0.40,-0.04,0.54,-0.68,-0.72,1.30,1.32,0.00,0.00,0.00,0.00,-0.61,1.32,0,-0.71,-0.71
7,0.79,-0.29,-1.03,-0.10,1.30,0.99,-0.40,-1.31,0.68,0.32,-1.58,-1.73,-1.70,-0.62,1.12,-0.77,-1.18,-0.13,-0.63,-1.03,0.48,0.32,-1.82,-1.46,-1.45,1.31,-1.28,0.00,-1.28,1.32,-1.28,0.00,-1.28,0.46,-1.34,0.92,1.72,0.50,0.43,0.60,0.40,-0.04,0.54,-0.68,-0.72,1.30,1.32,0.00,0.00,0.00,0.00,-0.61,1.32,0,-0.71,1.41
8,0.79,-0.29,-1.03,-0.10,1.30,0.99,-0.40,-1.31,0.68,0.32,-1.58,-1.73,-1.70,-0.62,1.12,-0.77,-1.18,-0.13,-0.63,-1.03,0.48,0.32,-1.82,-1.46,-1.45,1.31,-1.28,0.00,-1.28,1.32,-1.28,0.00,-1.28,0.46,-1.34,0.92,1.72,0.50,0.43,0.60,0.40,-0.04,0.54,-0.68,-0.72,1.30,1.32,0.00,0.00,0.00,0.00,-0.61,1.32,0,1.41,-0.71
9,-1.32,1.02,1.31,1.14,-0.03,-0.18,1.09,0.59,-0.16,0.32,-1.16,1.19,1.15,-0.45,1.26,0.36,0.71,1.71,-0.45,-0.31,-1.97,0.32,-0.60,0.41,0.45,-0.59,-0.34,0.00,-0.34,-0.76,-0.34,0.00,-0.34,1.14,-1.06,0.50,-0.61,-1.73,1.15,0.82,1.33,-0.04,-0.23,1.57,1.48,-0.54,-0.77,0.00,0.00,0.00,0.00,-0.61,-0.76,0,-0.71,-0.71


In [46]:
initial_cat_df

,scheduled_departure_time,scheduled_arrival_time,Carrier Code,time_of_day,origin_airport,flight_status_prev
0,2024-04-10 14:55:00,2024-04-10 16:21:00,DL,Evening,JFK,early
1,2024-04-10 14:55:00,2024-04-10 16:21:00,DL,Evening,JFK,ontime
2,2024-04-10 14:55:00,2024-04-10 16:21:00,DL,Evening,JFK,late
3,2024-04-10 13:35:00,2024-04-10 16:25:00,B6,Evening,MCO,early
4,2024-04-10 13:35:00,2024-04-10 16:25:00,B6,Evening,MCO,ontime
5,2024-04-10 13:35:00,2024-04-10 16:25:00,B6,Evening,MCO,late
6,2024-04-10 19:59:00,2024-04-10 22:52:00,AA,Night,ORD,early
7,2024-04-10 19:59:00,2024-04-10 22:52:00,AA,Night,ORD,ontime
8,2024-04-10 19:59:00,2024-04-10 22:52:00,AA,Night,ORD,late
9,2024-04-11 14:55:00,2024-04-11 16:21:00,DL,Evening,JFK,early


In [47]:
set(X_train.columns.tolist())-set(initial_df.columns.tolist())

set()

In [48]:
set(initial_df.columns.tolist())-set(final_df.columns.tolist())

set()

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

models = {
    'logistic_regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42,n_estimators = 250,max_depth = 50, learning_rate= 1 ),
    'ada' : AdaBoostClassifier(random_state=42),
    'knn' : KNeighborsClassifier(n_neighbors=15),
    'dtree':  DecisionTreeClassifier(max_depth=7,random_state=42),
    'GNB':  GaussianNB(),
    'rf_tuned': RandomForestClassifier(random_state=42, n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_depth=20, criterion='gini', bootstrap=True)
}


In [59]:
for model_name, model in models.items():
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'{model_name} Accuracy: {accuracy:.2f}')
        
    except Exception as e:
        print(e)

/Users/saikeerthanpalavarapu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

logistic_regression Accuracy: 0.57


DecisionTreeClassifier(random_state=42)

Decision Tree Accuracy: 0.46


...

dtree Accuracy: 0.56


GaussianNB()

GNB Accuracy: 0.28


RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200,
                       random_state=42)

rf_tuned Accuracy: 0.57


### Here, Tuned Random forest classifier has better accuracy among other models. So, Considering tuned Random forest classifier for predictions

In [63]:
rf = models["rf_tuned"]

rf = rf.fit(X_train, y_train) 
rf.score(X_train, y_train) 

test_output = pd.DataFrame(rf.predict(X_test), index = X_test.index, columns = ['pred_Y'])

test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
print('Fraction of correct classification ')
print(round(rf.score(X_test, y_test),2))


0.9430178268251274

,pred_Y,arrival_status
6686,early,early
4080,late,early
4782,late,early
8188,early,late
2847,early,late


Fraction of correct classification 
0.57


In [64]:
initial_pred_df = pd.DataFrame(rf.predict(initial_df), index = initial_df.index, columns = [f'initial_pred'])
final_pred_df = pd.DataFrame(rf.predict(final_df), index = initial_df.index, columns = [f'final_pred'])


In [65]:
pd.concat([initial_cat_df, initial_pred_df], axis=1)

,scheduled_departure_time,scheduled_arrival_time,Carrier Code,time_of_day,origin_airport,flight_status_prev,initial_pred
0,2024-04-10 14:55:00,2024-04-10 16:21:00,DL,Evening,JFK,early,early
1,2024-04-10 14:55:00,2024-04-10 16:21:00,DL,Evening,JFK,ontime,early
2,2024-04-10 14:55:00,2024-04-10 16:21:00,DL,Evening,JFK,late,early
3,2024-04-10 13:35:00,2024-04-10 16:25:00,B6,Evening,MCO,early,early
4,2024-04-10 13:35:00,2024-04-10 16:25:00,B6,Evening,MCO,ontime,early
5,2024-04-10 13:35:00,2024-04-10 16:25:00,B6,Evening,MCO,late,early
6,2024-04-10 19:59:00,2024-04-10 22:52:00,AA,Night,ORD,early,early
7,2024-04-10 19:59:00,2024-04-10 22:52:00,AA,Night,ORD,ontime,early
8,2024-04-10 19:59:00,2024-04-10 22:52:00,AA,Night,ORD,late,early
9,2024-04-11 14:55:00,2024-04-11 16:21:00,DL,Evening,JFK,early,early


In [66]:
pd.concat([final_cat_df, final_pred_df], axis=1)

,scheduled_departure_time,scheduled_arrival_time,Carrier Code,time_of_day,origin_airport,flight_status_prev,final_pred
0,2024-04-19 14:55:00,2024-04-19 16:21:00,DL,Evening,JFK,early,early
1,2024-04-19 14:55:00,2024-04-19 16:21:00,DL,Evening,JFK,ontime,early
2,2024-04-19 14:55:00,2024-04-19 16:21:00,DL,Evening,JFK,late,early
3,2024-04-19 13:35:00,2024-04-19 16:25:00,B6,Evening,MCO,early,early
4,2024-04-19 13:35:00,2024-04-19 16:25:00,B6,Evening,MCO,ontime,early
5,2024-04-19 13:35:00,2024-04-19 16:25:00,B6,Evening,MCO,late,early
6,2024-04-19 19:59:00,2024-04-19 22:52:00,AA,Night,ORD,early,early
7,2024-04-19 19:59:00,2024-04-19 22:52:00,AA,Night,ORD,ontime,early
8,2024-04-19 19:59:00,2024-04-19 22:52:00,AA,Night,ORD,late,early
9,2024-04-20 14:55:00,2024-04-20 16:21:00,DL,Evening,JFK,early,early
